In [1]:
import os
import sys
import random
import argparse

In [7]:
sampleList = [100, 200]
randomList = random.choices(
  sampleList, cum_weights=(1,100), k=1)
  
print(randomList)

[200]


In [44]:
"""
load rule from file
first we need to read each line of the grammar.gr, skip empty lines and lines start with #
split each line with 	, get count, key, value, then split the value by space, get the value list, such as NP VP
if the key is not in rules, rules[key]=[[value_list],[count]]
else:
    rules[key][0].append(value_list)
    rules[key][1].append(count)


we need a dictionary/hashmap: rules={key:[options,weight for each option]}

"""

def load_rules_from_file(grammar_file):
    rules = {}
    with open(grammar_file) as f:
        lines = f.read().splitlines()
        for line in lines:
            if not line.strip() or line.startswith("#"):
                continue
            else:
                #print(line)
                line.splitlines() 
                count,key,values = line.split("\t")
                value_list = values.split(" ") 
                #for case 1	ROOT	is it true that S ?     # mixing terminals and nonterminals is ok
                #print(key,value_list)
                for i in range(len(value_list)):
                    if value_list[i]=='':
                        value_list = value_list[:i]
                        break
                
                if key not in rules:
                    rules[key]=[[value_list],[int(count)]]
                else:
                    rules[key][0].append(value_list)
                    rules[key][1].append(int(count))
    return rules
    
    
    
grammar_file = "grammar.gr"
rules = load_rules_from_file(grammar_file)
print(rules)

{'ROOT': [[['S', '.'], ['S', '!'], ['is', 'it', 'true', 'that', 'S', '?']], [1, 1, 1]], 'S': [[['NP', 'VP']], [1]], 'VP': [[['Verb', 'NP']], [1]], 'NP': [[['Det', 'Noun'], ['NP', 'PP']], [1, 1]], 'PP': [[['Prep', 'NP']], [1]], 'Noun': [[['Adj', 'Noun'], ['president'], ['sandwich'], ['pickle'], ['chief', 'of', 'staff'], ['floor']], [1, 1, 1, 1, 1, 1]], 'Verb': [[['ate'], ['wanted'], ['kissed'], ['understood'], ['pickled']], [1, 1, 1, 1, 1]], 'Det': [[['the'], ['a'], ['every']], [1, 1, 1]], 'Adj': [[['fine'], ['delicious'], ['perplexed'], ['pickled']], [1, 1, 1, 1]], 'Prep': [[['with'], ['on'], ['under'], ['in']], [1, 1, 1, 1]]}


In [78]:
"""
sample use a recursion/dfs
output=""
def dfs(node):
    #base case the node is leaf node/terminal symbols
    append the node to the output list
    
    #otherwise, sample one possible structure
    childrens = random.choices(rules[node][0], cum_weights=rules[node][1], k=1)
    for child in childrens:
        dfs(child)
    

dfs(node = "root")

"""
output=[]
max_expansions = 10
expansion = 0 # global variables
def dfs(symbol):
    print(expansion)
    #if exceed the maximal number of nonterminals
    if expansion > max_expansions:
        output.append("...")
        return
    
    #base case
    if symbol not in rules:
        output.append(symbol)
        return 
    #otherwise
    
    childrens = random.choices(rules[symbol][0], weights=rules[symbol][1], k=1)[0]
    for child in childrens:
        expansion = expansion+ 1
        dfs(child)
start_symbol = 'ROOT'
dfs(start_symbol)       
print(output)

UnboundLocalError: local variable 'expansion' referenced before assignment

In [30]:
with open(grammar_file) as f:
    lines = f.read().splitlines() 
print(lines)

['# Symbols in the grammar are case-sensitive.', '# ', '# This grammar uses a convention that', '#    - terminals are usually lowercase  (president)', '#    - preterminals are capitalized     (Noun)', '#    - other nonterminals are all-caps  (NP)', '# ', '# This convention just makes grammars more readable to humans.  Thus:', '#', '#    - When *you* are writing grammars in questions 3 and 6, you should ', '#      follow this convention unless you have a good reason not to.  ', '#', "#    - But your *program* should still work with grammars that don't", '#      follow this convention.  So how can your program reliably tell', '#      the difference between terminal and nonterminal symbols?  If', '#      there is at least one rule for rewriting a symbol, then that', '#      symbol is a nonterminal and should be rewritten.', '#######################', '', '# Rules for creating full sentences.', '', '1\tROOT\tS .', '1\tROOT\tS !', '1\tROOT\tis it true that S ?     # mixing terminals and non

In [126]:
value = "is it true that S ?     # mixing terminals and nonterminals is ok"
value.split(" ")

['is',
 'it',
 'true',
 'that',
 'S',
 '?',
 '',
 '',
 '',
 '',
 '#',
 'mixing',
 'terminals',
 'and',
 'nonterminals',
 'is',
 'ok']

In [ ]:
    #helper function,combine nonterminal 
    #['is', 'it', 'true', 'that', 'S', '?'] --> ['is it true that', 'S', '?'], count stay unchanged
    def comb(self,value_list):
        
        comb_value_list = [value_list[0]]
        flag = True if values_list[0] not in self.values else False
        for i in range(1,len(values_list)):
            if values_list[i] not in self.rules:
                #
                if not flag:
                    flag=True
                    comb_value_list.appned(values_list[i])
                else:
                    #comb
                    comb_value_list[-1] = comb_value_list[-1]+' '+ values_list[i]
            else
                flag=False
                comb_value_list.appned(values_list[i])
            
        return comb_value_list
                
    def combNonterminals(self):
        """
        iterate through each item of self.rules, 
        
        """
        for key,vc in self.rules:
            values,count =vc[0],vc[1]
            #self.rules[key][0][i] = []
            for i in range(len(values)):
                value_list = values[i]
                comb_value_list = self.comb(value_list)
                self.rules[key][0][i] = comb_value_list

In [172]:
class Grammar:
    def __init__(self, grammar_file):
        """
        Context-Free Grammar (CFG) Sentence Generator

        Args:
            grammar_file (str): Path to a .gr grammar file
        
        Returns:
            self
        """
        # Parse the input grammar file
        self.rules = None
        self._load_rules_from_file(grammar_file)
    
    

    def _load_rules_from_file(self, grammar_file):
        """
        Read grammar file and store its rules in self.rules

        Args:
            grammar_file (str): Path to the raw grammar file 
        """
        """
        
        1.first we need to read each line of the grammar.gr, skip empty lines and lines start with #
        2,split each line with 	, get count, key, value, then split the value by space, get the value list, such as NP VP
        3.if the key is not in rules, rules[key]=[[value_list],[count]]
        else:
            rules[key][0].append(value_list)
            rules[key][1].append(count)


        we need a dictionary/hashmap: rules={key:[options,weight for each option]}

        """
        self.rules = {}
        with open(grammar_file) as f:
            lines = f.read().splitlines()
            for line in lines:
                if not line.strip() or line.startswith("#"):
                    continue
                else:
                    #print(line)
                    line.splitlines() 
                    count,key,values = line.split("\t")
                    value_list = values.split(" ") 
                    #for case 1	ROOT	is it true that S ?     # mixing terminals and nonterminals is ok
                    for i in range(len(value_list)):
                        if value_list[i]==''or value_list[i]=='#':
                            value_list = value_list[:i]
                            break
                            
                    
                    if key not in self.rules:
                        self.rules[key]=[[value_list],[int(count)]]
                    else:
                        self.rules[key][0].append(value_list)
                        self.rules[key][1].append(int(count))
        #comb Nonterminal items in self.rule
        self.combNonterminals()
        #return self.rules
        #raise NotImplementedError
        
    #helper function,combine nonterminal for each list
    #['is', 'it', 'true', 'that', 'S', '?'] --> ['is it true that', 'S', '?'], count stay unchanged
    def comb(self,value_list):
        
        comb_value_list = [value_list[0]]
        flag = True if value_list[0] not in self.rules else False
        
        for i in range(1,len(value_list)):
            if value_list[i] not in self.rules:
                #
                if not flag:
                    flag=True
                    comb_value_list.append(value_list[i])
                else:
                    #comb
                    comb_value_list[-1] = comb_value_list[-1]+' '+ value_list[i]
            else:
                flag=False
                comb_value_list.append(value_list[i])
            
        return comb_value_list
    
    #comb nonterminals for the all hashtable
    def combNonterminals(self):
        """
        iterate through each item of self.rules, 
        
        """
        for key in self.rules:
            values=self.rules[key][0] #count remain unchanged
            #self.rules[key][0][i] = []
            for i in range(len(values)):
                value_list = values[i]
                comb_value_list = self.comb(value_list)
                self.rules[key][0][i] = comb_value_list
                
    def sample(self, derivation_tree, max_expansions, start_symbol):
        """
        Sample a random sentence from this grammar

        Args:
            derivation_tree (bool): if true, the returned string will represent 
                the tree (using bracket notation) that records how the sentence 
                was derived
                               
            max_expansions (int): max number of nonterminal expansions we allow

            start_symbol (str): start symbol to generate from

        Returns:
            str: the random sentence or its derivation tree
        """
        self.output=[]
        self.expansion = 0 # global variables
        self.max_expansions = max_expansions
        if derivation_tree:
            self.right=0 #number of right parenthese
            self.dfs_tree(start_symbol)
            self.output.append(")"*self.right)
        else:
            self.dfs(start_symbol)
        #return self.output
        return " ".join(self.output)
    
    def dfs(self,symbol):
        print(self.expansion,symbol)
        #if exceed the maximal number of nonterminals
        if self.expansion > self.max_expansions:
            self.output.append("...")
            return

        #base case
        if symbol not in rules:
            self.output.append(symbol)
            return 
        #otherwise
        children = random.choices(self.rules[symbol][0], weights=self.rules[symbol][1], k=1)[0]
        self.expansion+= 1
        for child in children:
            #self.expansion+= 1
            self.dfs(child)
            
    def dfs_tree(self,symbol):
        print(self.expansion,symbol)
        #if exceed the maximal number of nonterminals
        if self.expansion > self.max_expansions:
            self.output.append("...)")
            self.right-=1
            return

        #base case
        if symbol not in rules:
            self.output.append(symbol+")")
            self.right-=1
            return 
        #otherwise
        self.output.append("(" +symbol)
        self.right+=1
        children = random.choices(self.rules[symbol][0], weights=self.rules[symbol][1], k=1)[0]
        self.expansion+= 1
        for child in children:
            #self.expansion+= 1
            self.dfs_tree(child)

        #raise NotImplementedError

In [173]:
grammar_file = "grammar.gr"
derivation_tree = True
max_expansions =15
start_symbol = "ROOT"
grammar = Grammar(grammar_file)
#grammer._load_rules_from_file(grammar_file)
print(grammar.rules)
sentence = grammar.sample(derivation_tree, max_expansions, start_symbol)
print(sentence)
if derivation_tree:
    prettyprint_path = os.path.join(os.getcwd(), 'prettyprint')
    t = os.system(f"echo '{sentence}' | perl {prettyprint_path}")

{'ROOT': [[['S', '.'], ['S', '!'], ['is it true that', 'S', '?']], [1, 1, 1]], 'S': [[['NP', 'VP']], [1]], 'VP': [[['Verb', 'NP']], [1]], 'NP': [[['Det', 'Noun'], ['NP', 'PP']], [1, 1]], 'PP': [[['Prep', 'NP']], [1]], 'Noun': [[['Adj', 'Noun'], ['president'], ['sandwich'], ['pickle'], ['chief of staff'], ['floor']], [1, 1, 1, 1, 1, 1]], 'Verb': [[['ate'], ['wanted'], ['kissed'], ['understood'], ['pickled']], [1, 1, 1, 1, 1]], 'Det': [[['the'], ['a'], ['every']], [1, 1, 1]], 'Adj': [[['fine'], ['delicious'], ['perplexed'], ['pickled']], [1, 1, 1, 1]], 'Prep': [[['with'], ['on'], ['under'], ['in']], [1, 1, 1, 1]]}
0 ROOT
1 is it true that
1 S
2 NP
3 Det
4 a
4 Noun
5 sandwich
5 VP
6 Verb
7 wanted
7 NP
8 NP
9 Det
10 the
10 Noun
11 pickle
11 PP
12 Prep
13 with
13 NP
14 NP
15 Det
16 a
16 Noun
16 PP
16 ?
(ROOT is it true that) (S (NP (Det a) (Noun sandwich) (VP (Verb wanted) (NP (NP (Det the) (Noun pickle) (PP (Prep with) (NP (NP (Det ...) ...) ...) ...) )))))


In [171]:
prettyprint_path

'/Users/meililiu/Desktop/Course/601.665 NLP/homework/hw-grammar/prettyprint'

In [116]:
s[::-1]

['2', '1']

In [117]:
s[0:2]

['1', '2']

In [123]:
s = "aba"
s==s[::-1]

True

In [124]:
[1,2]+[3]

[1, 2, 3]

In [3]:
a= tuple(["1","2"])

In [4]:
a

('1', '2')

In [5]:
b=3"

In [7]:
a+tuple(b)

('1', '2', '3')

In [9]:
a= set()

In [10]:
a.add(1)

In [11]:
a.add(2)

In [13]:
a.update([3,2])

In [14]:
a

{1, 2, 3}

In [4]:
s = 'aabbbccdddddee'
from collections import Counter
counter = Counter(s)
sorted(counter.values(), reverse=True)

[5, 3, 2, 2, 2]

In [8]:
counter = counter.most_common()

In [11]:
counter[0][0]

'd'

In [12]:
sum(counter[1:][1])

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [14]:
counter[1][1]

3

In [15]:
s ="abdcs"
s[2]

'd'

In [18]:
a = "a"*3

In [19]:
a

'aaa'

In [21]:
a =set((1,2,3))

In [22]:
a

{1, 2, 3}

TypeError: 'set' object is not subscriptable